In [6]:
!pip install webvtt-py
!pip install opencv-python
!pip install yt_dlp
!pip install webvtt-py
!pip install opencv-python

import pandas as pd
import importlib
import yt_dlp as youtube_dl
import requests
import cv2
import numpy as np
import os
import webvtt
import json
import concurrent.futures
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.1/170.1 kB 3.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 38.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.4 MB/s eta 0:00:00:00:01


In [7]:
# Load CSV containing all links
df = pd.read_csv('/kaggle/input/youtube-sign-lang/youtube_videos_ish.csv')
df.head()

,URL
0,https://www.youtube.com/watch?v=dgJzKm2Icj0
1,https://www.youtube.com/watch?v=8eu7yoQ6hMY
2,https://www.youtube.com/watch?v=xPvqJmezApM
3,https://www.youtube.com/watch?v=fjJYlT2Wxpk
4,https://www.youtube.com/watch?v=rMD_0imguKg


In [8]:
# Define Video Downloader

def download_video_with_subtitles(video_url, output_path, subtitle_language='en'):
    ydl_opts = {
        'outtmpl': output_path,  # Path where video will be saved
        'format': 'best',  # Download the best quality format
        'subtitleslangs': [subtitle_language],  # Subtitle language
        'writeautomaticsub': True,  # Download automatic subtitles
        'writesubtitles': True,  # Download subtitles
        'subtitlesformat': 'srt',  # Format of subtitles
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([video_url])

# Example usage:

# video_url = videos[0]
# output_path = 'videos/video_1.mp4'
# download_video_with_subtitles(video_url, output_path)

In [9]:
# Download required files for YOLOV3 human detection model

res = requests.get('https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names')

with open('coco.names', 'wb') as f:
    f.write(res.content)

!git clone https://github.com/pjreddie/darknet
!wget https://raw.githubusercontent.com/smarthomefans/darknet-test/master/yolov3-tiny.weights

Cloning into 'darknet'...
remote: Enumerating objects: 5955, done.
remote: Total 5955 (delta 0), reused 0 (delta 0), pack-reused 5955
Receiving objects: 100% (5955/5955), 6.37 MiB | 13.68 MiB/s, done.
Resolving deltas: 100% (3932/3932), done.
--2024-07-30 20:52:20--  https://raw.githubusercontent.com/smarthomefans/darknet-test/master/yolov3-tiny.weights
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: 'yolov3-tiny.weights'

yolov3-tiny.weights 100%[===================>]  33.79M  61.8MB/s    in 0.5s    

2024-07-30 20:52:24 (61.8 MB/s) - 'yolov3-tiny.weights' saved [35434956/35434956]



In [10]:
classes = []
with open('coco.names', 'r') as f:
    classes = [x.replace('\n', '') for x in f.readlines()]
    
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [11]:
yolo_cfg_path = '/kaggle/working/darknet/cfg/yolov3-tiny.cfg'  # Update with the correct path
yolo_weights_path = '/kaggle/working/yolov3-tiny.weights'  # Update with the correct path

import cv2
import numpy as np

# Load YOLOv4 model
net = cv2.dnn.readNet(yolo_weights_path, yolo_cfg_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Function to process batch of frames
def process_batch(frames):
    batch_size = len(frames)
    height, width = frames[0].shape[:2]
#     print(len(frames), frames[0].shape)
    # Create a batch of input images
    blob = cv2.dnn.blobFromImages(frames, 0.00392, (416, 416), (0, 0, 0), swapRB=True, crop=False)
    net.setInput(blob)
    
    # Run inference
    outs = net.forward(output_layers)[0]
#     print(len(outs))
#     print(outs[0].shape)
    all_boxes = []
    all_confidences = []
    all_class_ids = []

    # Process detections for each frame in the batch
    for i in range(batch_size):
        frame_boxes = []
        frame_confidences = []
        frame_class_ids = []
        for detection in outs[i]:
            scores = detection[5:]
#             print(scores.shape)
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and class_id == 0:  # Class ID 0 is 'person'
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                frame_boxes.append([x, y, w, h])
                frame_confidences.append(float(confidence))
                frame_class_ids.append(class_id)
        
        all_boxes.append(frame_boxes)
        all_confidences.append(frame_confidences)
        all_class_ids.append(frame_class_ids)

    return all_boxes, all_confidences, all_class_ids

In [12]:
import matplotlib.pyplot as plt

# Load multiple frames (e.g., from a video)
cap = cv2.VideoCapture('video.mp4')
batch_size = 10
frames = []
outframes = []
x = 0

def crop(img):

    y_start = 0
    y_end = img.shape[0]
    x_start = 250
    x_end = img.shape[1]
    
    cropped_image = img[y_start:y_end, x_start:x_end]
    return cropped_image



In [13]:
import re
import difflib

def clean_text(text):
    """Removes unnecessary characters and normalizes whitespace."""
    text = re.sub(r'\n', ' ', text)  # Replace newline characters with spaces
    text = re.sub(r'[^\w\s]', ' ', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.strip().lower()  # Convert to lowercase for uniformity

def longest_common_substring(s1, s2):
    matcher = difflib.SequenceMatcher(None, s1, s2)
    match = matcher.find_longest_match(0, len(s1), 0, len(s2))
    if match.size != 0:
        return s1[match.a: match.a + match.size]
    return ""

def merge_subtitles(subtitles):
    filtered = []
    current_end = subtitles[0][0]
    current_string = ''

    for i in range(len(subtitles)):
        start, end, text = subtitles[i]
        
        if(end - start <= 2):
            continue
              
        text = clean_text(text)
        lcs = longest_common_substring(current_string, text)
        if lcs:
            # Check if the current string ends with LCS and the new text starts with LCS
            if current_string.endswith(lcs) and text.startswith(lcs):
                text = text[len(lcs):].strip()
                current_string += ' ' + text
                
        if(text != ''):
            filtered.append((current_end, end, text))
            current_string = text
            current_end = end
    
    filtered.append((current_end, end, current_string))
    current_end = max(current_end, end)
    return filtered



def convert_to_seconds(x):
    h, m, s = [float(ti) for ti in x.split(':')]
    return h * 3600 + m * 60 + s

def parse_vtt(vtt_file):
    clips = []
    for caption in webvtt.read(vtt_file):
        start_time = convert_to_seconds(caption.start)
        end_time = convert_to_seconds(caption.end)
        content = caption.text
        clips.append((start_time, end_time, content))
    
    return merge_subtitles(clips)

def extract_clip(video_path, start_time, end_time):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    
    frames = []
    for frame_num in range(start_frame, end_frame):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    
    cap.release()
    return frames

In [14]:
def process_frames(frames):
    out_frames = []
    results = []
    batch_size = 20
    boxes, confidences, class_ids = [], [], []
    
    for i in range(0, len(frames), batch_size):
        input_batch = frames[i:i+batch_size]
        if len(input_batch) > 1:
            batch_boxes, batch_confidences, batch_class_ids = process_batch(input_batch)
            boxes.extend(batch_boxes)
            confidences.extend(batch_confidences)
            class_ids.extend(batch_class_ids)

    for j in range(len(frames)):
        if j < len(boxes):
            for i in range(len(boxes[j])):
                if confidences[j][i] < 0.3:
                    continue
                
                x, y, w, h = boxes[j][i]
                if w > 0.3 * frames[j].shape[1] and x > 0.4 * frames[j].shape[1]:
                    cropped_image = crop(frames[j])
                    out_frames.append(cropped_image)

    return out_frames


In [15]:
def save_clip(frames, output_path):
    height, width, layers = frames[0].shape
    size = (width, height)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), 20, size)
    
    for frame in frames:
        resized_frame = cv2.resize(frame, size)
        out.write(resized_frame)
    
    out.release()

In [16]:
def process_clip(video_path, start_time, end_time, output_folder, file_name, subtitles, content):
    frames = extract_clip(video_path, start_time, end_time)
    processed_frames = process_frames(frames)
    if(len(processed_frames) > 0.7*len(frames)):   
        print(file_name)
        output_path = f'{output_folder}/{file_name}'
        save_clip(processed_frames, output_path)
        subtitles[file_name] = content
        
def process_video_subtitles(video_path, vtt_file, output_folder, video_num, subtitles):
    clips = parse_vtt(vtt_file)
    dumped = 0
    for i, (start_time, end_time, content) in enumerate(clips):
        if start_time != end_time:
            file_name = f'clip_{video_num}_{i}.mp4'
            process_clip(video_path, start_time, end_time, output_folder, file_name, subtitles, content)


In [17]:
# !rm /kaggle/working/clips/*.mp4

In [18]:
output_folder = '/kaggle/working/clips'
os.makedirs(output_folder, exist_ok=True)

subtitles = {}

for video_num, url in enumerate(df['URL'][450:500]):
    try:
        print("Saving file: ", video_num)
        video_path = f'/kaggle/tmp/{video_num}.mp4'
        vtt_path = f'/kaggle/tmp/{video_num}.en.vtt'
        download_video_with_subtitles(url, video_path)
        process_video_subtitles(video_path, vtt_path, output_folder, video_num, subtitles)
    except Exception as e:
        print(f"Error processing video {video_num}: {e}")

with open('subtitles.json', 'w') as f:
    json.dump(subtitles, f)

Saving file:  0
[youtube] Extracting URL: https://www.youtube.com/watch?v=dTI9GUdzgsQ
[youtube] dTI9GUdzgsQ: Downloading webpage


[youtube] dTI9GUdzgsQ: Downloading ios player API JSON
[youtube] dTI9GUdzgsQ: Downloading player 250a2ff7
[youtube] dTI9GUdzgsQ: Downloading web player API JSON
[youtube] dTI9GUdzgsQ: Downloading m3u8 information
[info] dTI9GUdzgsQ: Downloading 1 format(s): 18
[info] There are no subtitles for the requested languages
[download] Destination: /kaggle/tmp/0.mp4
[download] 100% of    3.15MiB in 00:00:01 at 2.28MiB/s   
Error processing video 0: [Errno 2] No such file or directory: '/kaggle/tmp/0.en.vtt'


In [19]:
!ls /kaggle/tmp

0.mp4
